In [1]:
import numpy as np
import math 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Activation 
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
pip install tensorflow-privacy==0.1.0

     |████████████████████████████████| 81kB 5.5MB/s 


In [ ]:
import tensorflow as tf

from privacy.analysis import privacy_ledger
from privacy.dp_query import gaussian_query

def make_optimizer_class(cls):
  """Constructs a DP optimizer class from an existing one."""
  parent_code = tf.optimizers.Optimizer._compute_gradients.__code__
  child_code = cls._compute_gradients.__code__
  if child_code is not parent_code:
    tf.logging.warning(
        'WARNING: Calling make_optimizer_class() on class %s that overrides '
        'method compute_gradients(). Check to ensure that '
        'make_optimizer_class() does not interfere with overridden version.',
        cls.__name__)

  class DPOptimizerClass(cls):
    """Differentially private subclass of given class cls."""

    def __init__(
        self,
        dp_sum_query,
        num_microbatches=None,
        unroll_microbatches=False,
        *args,
        **kwargs):
      """Initialize the DPOptimizerClass.
      Args:
        dp_sum_query: DPQuery object, specifying differential privacy
          mechanism to use.
        num_microbatches: How many microbatches into which the minibatch is
          split. If None, will default to the size of the minibatch, and
          per-example gradients will be computed.
        unroll_microbatches: If true, processes microbatches within a Python
          loop instead of a tf.while_loop. Can be used if using a tf.while_loop
          raises an exception.
      """
      super(DPOptimizerClass, self).__init__(*args, **kwargs)
      ###### accountant + sanitizer ######
      self._dp_sum_query = dp_sum_query
      ######
      self._num_microbatches = num_microbatches
      self._global_state = self._dp_sum_query.initial_global_state()
      self._unroll_microbatches = unroll_microbatches

    def compute_gradients(self, loss, var_list, gate_gradients=None, aggregation_method=None, colocate_gradients_with_ops=False, grad_loss=None, gradient_tape=None):
      if not gradient_tape:
        raise ValueError('A tape needs to be passed.')

      vector_loss = loss()
      if self._num_microbatches is None:
        self._num_microbatches = tf.shape(vector_loss)[0]
      sample_state = self._dp_sum_query.initial_sample_state(var_list)
      microbatches_losses = tf.reshape(vector_loss, [self._num_microbatches, -1])
      sample_params = (self._dp_sum_query.derive_sample_params(self._global_state))

      for idx in range(self._num_microbatches):
        ###### compute gradient ######
        microbatch_loss = tf.reduce_mean(tf.gather(microbatches_losses, [idx]))
        grads = gradient_tape.gradient(microbatch_loss, var_list)
        ######

        ###### accountant ######
        sample_state = self._dp_sum_query.accumulate_record(sample_params, sample_state, grads)
        ######

      ###### sanitizer ######
      grad_sums, self._global_state = (self._dp_sum_query.get_noised_result(sample_state, self._global_state)) 
      ######

      def normalize(v):
        return v / tf.cast(self._num_microbatches, tf.float32)

      final_grads = tf.nest.map_structure(normalize, grad_sums)

      grads_and_vars = list(zip(final_grads, var_list))
      return grads_and_vars

  return DPOptimizerClass


def make_gaussian_optimizer_class(cls):
  """Constructs a DP optimizer with Gaussian averaging of updates."""

  class DPGaussianOptimizerClass(make_optimizer_class(cls)):
    """DP subclass of given class cls using Gaussian averaging."""

    def __init__(self, l2_norm_clip, noise_multiplier, num_microbatches=None, ledger=None, unroll_microbatches=False, *args, **kwargs):
      dp_sum_query = gaussian_query.GaussianSumQuery(l2_norm_clip, l2_norm_clip * noise_multiplier)

      if ledger:
        dp_sum_query = privacy_ledger.QueryWithLedger(dp_sum_query, ledger=ledger)

      super(DPGaussianOptimizerClass, self).__init__(dp_sum_query, num_microbatches, unroll_microbatches, *args, **kwargs)

    @property
    def ledger(self):
      return self._dp_sum_query.ledger

  return DPGaussianOptimizerClass

DPAdagradOptimizer = make_optimizer_class(tf.optimizers.Adagrad)
DPAdamOptimizer = make_optimizer_class(tf.optimizers.Adam)
DPGradientDescentOptimizer = make_optimizer_class(tf.optimizers.SGD)

DPAdagradGaussianOptimizer = make_gaussian_optimizer_class(tf.optimizers.Adagrad)
DPAdamGaussianOptimizer = make_gaussian_optimizer_class(tf.optimizers.Adam)
DPGradientDescentGaussianOptimizer = make_gaussian_optimizer_class(tf.optimizers.SGD)

model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8, strides=2, padding='same', activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4, strides=2,  padding='valid', activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(10)
  ])

optimizer = DPGradientDescentGaussianOptimizer(
        l2_norm_clip=0.1,
        noise_multiplier=0.1,
        num_microbatches=10,
        learning_rate=0.01)
    # Compute vector of per-example loss rather than its mean over a minibatch.
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.compat.v1.losses.Reduction.NONE)


  # Compile model with Keras
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

  # Train model with Keras

fit_history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels), batch_size=50, verbose=True)

val_acc_history = fit_history.history['val_accuracy']

Epoch 1/10


ValueError: ignored